<a href="https://colab.research.google.com/github/jpellicott/ner_and_clustering/blob/main/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 2.5 MB/s 
     |████████████████████████████████| 5.2 MB 37.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 5.1 MB/s 
     |████████████████████████████████| 88 kB 8.1 MB/s 
     |████████████████████████████████| 4.7 MB 73.9 MB/s 
     |████████████████████████████████| 1.3 MB 62.5 MB/s 
     |████████████████████████████████| 101 kB 11.4 MB/s 
     |████████████████████████████████| 636 kB 85.8 MB/s 
     |████████████████████████████████| 6.6 MB 57.7 MB/s 
     |████████████████████████████████| 1.1 MB 85.1 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2340326 sha256=27e059ddf3611606380a387e2806ee29cbe49fa17e0ee38113ef438dc6874719
  Stored in directory: /root/.cache

In [3]:
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer


In [5]:
data_path = "/content/drive/MyDrive/glg_data/all-the-news-2-1.csv"

In [6]:
data = pd.read_csv(
    data_path)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
data.shape

(2688879, 12)

In [8]:
data['day_date'] = pd.to_datetime(data['date'])

In [9]:
max_day = data['day_date'].max()

In [10]:
min_day = max_day - pd.DateOffset(months = 12)

In [11]:
max_day

Timestamp('2020-04-02 00:00:00')

In [12]:
min_day

Timestamp('2019-07-02 00:00:00')

In [13]:
data = data[data['day_date'] >= min_day]

In [14]:
#data = data[0:500_000]

In [15]:
#data = data[0:1_000_000]

In [16]:
#data

In [17]:
#from the project "Effort can be limited to mining a single paragraph of text from these articles only)""
# def get_first_paragraph(tweet):
#   try:
#     tweet = [re.search(r'\n\n((.*\n)*?\n)', tweet).group(1).replace('\n', '')]
#   except:
#     tweet = tweet
#     return tweet

In [18]:
data['article'] = data['title'] + ' ' + data['article']
data['article'] = data['article'].str.lower()

In [19]:
data['article'] = data['article'].str[:150]

In [20]:
def remove_non_alphateic(tweet):
    tweet = re.sub('[^a-zA-Z ]', '',str(tweet))
    return tweet

In [21]:
def remove_short_words(tweet):
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    return tweet

In [22]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
#stop words is just a list, appending other words we didn't feel added value after initial runs
stop_words.append('said')
stop_words.append('would')
stop_words.append('also')
stop_words.append('could')
stop_words.append('even')
stop_words.append('told')
stop_words.append('says')
stop_words.append('make')
stop_words.append('dont')
stop_words.append('thats')



In [24]:
stopwords_dict = Counter(stop_words)


In [25]:
data['article'] = data['article'].apply(remove_non_alphateic)

In [26]:
data['article'] = data['article'].apply(remove_short_words)

In [27]:
data['article'] = data['article'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_dict)]))

In [28]:
#data = data.sample(frac=0.90)

In [29]:
data.shape

(597478, 13)

In [30]:
articles = data['article'].to_list()

In [31]:
dates = data['date'].apply(lambda x: pd.Timestamp(x)).to_list()


In [32]:
del data


In [33]:
#using custom paramters to lower memory usage
vectorizer_model = CountVectorizer(ngram_range=(1, 2), min_df=.015, max_df=.7, max_features=10000)

In [34]:
from bertopic import BERTopic
topic_model = BERTopic(min_topic_size=500, 
                       n_gram_range=(1,2), 
                       low_memory = True,
                       verbose=True,
                       calculate_probabilities=False,
                       vectorizer_model = vectorizer_model,
                       #top_n_words=8
                       nr_topics = 'auto'
                       )
topics, _ = topic_model.fit_transform(articles)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/18672 [00:00<?, ?it/s]

2022-08-09 05:37:27,648 - BERTopic - Transformed documents to Embeddings
2022-08-09 05:47:31,019 - BERTopic - Reduced dimensionality
2022-08-09 05:50:52,716 - BERTopic - Clustered reduced embeddings
2022-08-09 05:52:22,089 - BERTopic - Reduced number of topics from 146 to 98


In [35]:
topic_model.save("/content/drive/MyDrive/glg/topic_model_12_months.pt",save_embedding_model=False)


/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [36]:
topic_model.save("/content/drive/MyDrive/glg/topic_model_with_embedding_12_months.pt",save_embedding_model=True)


In [37]:
topic_model.visualize_topics()


In [38]:
topic_model.visualize_barchart(top_n_topics=107)


In [39]:
topics_over_time = topic_model.topics_over_time(articles, topics, dates, nr_bins=20)


20it [00:19,  1.05it/s]


In [40]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)


In [42]:
freq = topic_model.get_topic_info()
freq.head(50)


,Topic,Count,Name
0,-1,251256,-1_democratic_court_german_amazon
1,0,69156,0_nan nan_nan__
2,1,30096,1_iphone_apple_shooting_google
3,2,23610,2_hong_kong_hong kong_korea
4,3,16752,3_oil_saudi_iran_syria
5,4,14610,4_brexit_johnson_boris_boris johnson
6,5,12458,5_brazil_brazils_brazilian_mexico
7,6,12130,6_ltd_mln_net_inc
8,7,10224,7_editorsnote_graf_scored_scored points
9,8,8929,8_tesla_electric_uber_musk


In [43]:
topic_nr = freq.iloc[6]["Topic"] # select a frequent topic
topic_model.get_topic(topic_nr)

[('brazil', 0.057789990212439954),
 ('brazils', 0.05666746688696806),
 ('brazilian', 0.042935334628864036),
 ('mexico', 0.0424211125456455),
 ('paulo', 0.04024410048867968),
 ('sao', 0.03992763267051388),
 ('sao paulo', 0.038075228047776695),
 ('brasilia', 0.03476453427007482),
 ('italys', 0.032269483823869544),
 ('mexico city', 0.03197973571092783)]